In [60]:
class Note:
    
    def __init__(self, channel, pitch, velocity, timestamp, duration, prev_note_buffer):
        self.channel = channel
        self.pitch = pitch
        self.velocity = velocity
        
        # in MIDI 'ticks'
        self.timestamp = timestamp
        self.duration = duration
        self.prev_note_buffer = prev_note_buffer
        
        # rounding values for prediction
        self.velocity_round = 40
        self.duration_round = 20000
        
    def rounded(self):
        return Note(self.channel, self.pitch, self.velocity - (self.velocity % self.velocity_round), \
                    self.timestamp, self.duration - (self.duration % self.duration_round), self.prev_note_buffer)
        
    def __eq__(self, other):
        return self.channel == other.channel and self.pitch == other.pitch and self.velocity == other.velocity and \
            self.duration == other.duration and self.prev_note_buffer == other.prev_note_buffer
        
    def __repr__(self):
        return f"[Note {self.channel} {self.pitch} {self.velocity} {self.duration}]"
        
    def __hash__(self):
        return hash(str(self))

In [63]:
from collections import defaultdict
from random import randint
from midiutil.MidiFile import MIDIFile

class MarkovGenerator:
    
    def __init__(self, order, note_lst):
        self.order = order
        self.note_lst = note_lst
        self.adj = defaultdict(list)
        
        self.build_graph()
        
    def build_graph(self):
        
        for i in range(len(note_lst)):
            
            for j in range(i+1, i+1+self.order):
                
                note_slice = tuple(note_lst[i:j])
                note_slice_rounded = tuple(note.rounded() for note in note_slice)
                
                if j >= len(note_lst):
                    # out of bounds
                    break
                    
                nxt_note = note_lst[j]
                self.adj[note_slice_rounded].append(nxt_note)
                
#         for k, v in self.adj.items():
#             print(f"key: {k}, val: {v}")
                
    def generate_notes(self, song_length):
        
        output_notes = self.note_lst[:self.order]
        match_freq = defaultdict(int)
        
        for i in range(song_length):
            
            start_idx = max(0, len(output_notes) - self.order)
            end_idx = min(len(output_notes), start_idx + self.order)
            
            while start_idx < end_idx:
                
                note_slice = tuple(output_notes[start_idx: end_idx])
                note_slice_rounded = tuple(note.rounded() for note in note_slice)
                
                # print(f"trying len {end_idx - start_idx}")
            
                if note_slice_rounded in self.adj:
                    
                    # found a match in our adjacency list
                    
                    # print("using", start_idx, end_idx, note_slice, output_notes[-3:])
                    
                    match_freq[len(note_slice_rounded)] += 1
                    
                    options = self.adj[note_slice_rounded]                    
                    output_notes.append(options[randint(0, len(options) - 1)])
                    
                    break
                
                start_idx += 1
                
        print(f"Order {self.order} Markov match frequency --> {sorted(match_freq.items())}")
        # print(f"Order {self.order} Markov notes: {output_notes}")
        return output_notes
    
    def generate_midi(self, song_length):
    
        notes = self.generate_notes(song_length)

        track = 0
        time = 0
        tempo = 60 # In BPM

        MyMIDI = MIDIFile(1, eventtime_is_ticks=True)
        MyMIDI.addTempo(track,time,tempo)
        
        cur_tick = 0
        note_off_map = {}

        for note in notes:
            
            cur_tick += note.prev_note_buffer
            
            if (note.channel, note.pitch) in note_off_map and note_off_map[(note.channel, note.pitch)] > cur_tick:
                continue
                
            note_off_map[(note.channel, note.pitch)] = cur_tick + note.duration
            
            MyMIDI.addNote(track, note.channel, note.pitch, cur_tick, note.duration, note.velocity)

        return MyMIDI

In [64]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile('the_real_folk_blues.mid')

note_set = set()
note_lst = []

open_notes = {}

counter = 0
cur_tick = 0
prev_note_on_tick = 0

for msg in mid.tracks[0]:
    
    cur_tick += msg.time
    
    if msg.type == 'note_on' and msg.velocity != 0:
       
        note = Note(msg.channel, msg.note, msg.velocity, cur_tick, 0, cur_tick - prev_note_on_tick)
        open_notes[(msg.channel, msg.note)] = note
        counter += 1
        
        prev_note_on_tick = cur_tick
        
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        
        note_id = (msg.channel, msg.note)
        
        if note_id in open_notes:
            
            open_notes[note_id].duration = cur_tick - open_notes[note_id].timestamp
            
            note_set.add(open_notes[note_id])
            note_lst.append(open_notes[note_id])
            
            open_notes.pop(note_id)
        
print(f"Total notes: {counter}")
print(f"Unique notes: {len(note_set)}")
print(f"Unique rounded notes: {len(set(note.rounded() for note in note_set))}")

note_lst.sort(key=lambda x: x.timestamp)

MAX_ORDER = 10
for order in range(1, MAX_ORDER + 1):
    
    generator = MarkovGenerator(order, note_lst)
    output_mid = generator.generate_midi(2000)

    with open(f"folk_blues_order_{order}_markov.mid", "wb") as output_file:
        output_mid.writeFile(output_file)
    
print('finished')

Total notes: 2100
Unique notes: 440
Unique rounded notes: 181
Order 1 Markov match frequency --> [(1, 2000)]
Order 2 Markov match frequency --> [(2, 2000)]
Order 3 Markov match frequency --> [(3, 2000)]
Order 4 Markov match frequency --> [(4, 2000)]
Order 5 Markov match frequency --> [(5, 2000)]
Order 6 Markov match frequency --> [(6, 2000)]
Order 7 Markov match frequency --> [(7, 2000)]
Order 8 Markov match frequency --> [(8, 2000)]
Order 9 Markov match frequency --> [(9, 2000)]
Order 10 Markov match frequency --> [(10, 2000)]
finished
